In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import gc
from prj.config import DATA_DIR


BASE_PATH = DATA_DIR


partition=5
train_ds = pl.concat([
    pl.scan_parquet(BASE_PATH / f'partition_id={i}' / 'part-0.parquet')
    for i in range(partition, partition + 1)
]).collect().sort('date_id', 'time_id', 'symbol_id')
features = [col for col in train_ds.columns if col.startswith('feature_')]
target_feature = 'responder_6'
X = train_ds.select(features).cast(pl.Float32).to_numpy()
y = train_ds[target_feature].cast(pl.Float32).to_numpy()
weights = train_ds['weight'].cast(pl.Float32).to_numpy()

partition=6
train_ds = pl.concat([
    pl.scan_parquet(BASE_PATH / f'partition_id={i}' / 'part-0.parquet')
    for i in range(partition, partition + 1)
]).collect().sort('date_id', 'time_id', 'symbol_id')
X_test = train_ds.select(features).cast(pl.Float32).to_numpy()
y_test = train_ds[target_feature].cast(pl.Float32).to_numpy()
test_weights = train_ds['weight'].cast(pl.Float32).to_numpy()

del train_ds
gc.collect()

X.shape, y.shape, X_test.shape, y_test.shape

FileNotFoundError: No such file or directory (os error 2): ...ojects/jane_street_forecasting/dataset/train.parquet/train/partition_id=5/part-0.parquet

In [ ]:
from prj.agents.factory import AgentsFactory

agent = AgentsFactory.build_agent({'agent_type': 'lgbm', 'n_seeds': 2})
agent.load(DATA_DIR.parent / 'saved_agents' / 'lgbm')
agent

Loading models, overwriting seeds


In [ ]:
# agent.train(X, y)

In [ ]:
agent.evaluate(X, y, weights=weights)

[LightGBM] [Warning] Unknown parameter: object
[LightGBM] [Warning] Unknown parameter: at
[LightGBM] [Warning] Unknown parameter: callbacks
[LightGBM] [Warning] Unknown parameter: 0x316534910>
[LightGBM] [Warning] Unknown parameter: object
[LightGBM] [Warning] Unknown parameter: at
[LightGBM] [Warning] Unknown parameter: callbacks
[LightGBM] [Warning] Unknown parameter: 0x316534df0>


{'r2_w': 0.04000995259244777,
 'mae_w': 0.5289949000538194,
 'mse_w': 0.703657078637363,
 'rmse_w': 0.8388427019634629}

In [ ]:
agent.evaluate(X_test, y_test, weights=test_weights)

[LightGBM] [Warning] Unknown parameter: object
[LightGBM] [Warning] Unknown parameter: at
[LightGBM] [Warning] Unknown parameter: callbacks
[LightGBM] [Warning] Unknown parameter: 0x316536920>
[LightGBM] [Warning] Unknown parameter: object
[LightGBM] [Warning] Unknown parameter: at
[LightGBM] [Warning] Unknown parameter: callbacks
[LightGBM] [Warning] Unknown parameter: 0x3165343a0>


{'r2_w': 0.008223989375221175,
 'mae_w': 0.5576402456658255,
 'mse_w': 0.7344692394108066,
 'rmse_w': 0.857011808209669}

In [ ]:
# agent.save(DATA_DIR.parent / 'saved_agents' / 'lgbm')